In [2]:
from loguru import logger

import pandas as pd

from slt_positional_bias.dataset import generate_merged_data_frame, sort_data_frame, store_df_as_parquet, load_parquet_as_df

df = generate_merged_data_frame()

df_sorted = sort_data_frame(df, 1, 39)
df_sorted

2025-08-09 18:03:59.793 | INFO     | slt_positional_bias.config:<module>:11 - PROJ_ROOT path is: C:\Users\marti\slt_group_2_positional_bias
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\marti\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


2025-08-09 18:04:02.253 | INFO     | slt_positional_bias.dataset:generate_data_frame:151 - file path exists
2025-08-09 18:04:08.589 | INFO     | slt_positional_bias.dataset:generate_data_frame:151 - file path exists
2025-08-09 18:04:08.603 | INFO     | slt_positional_bias.dataset:generate_data_frame:151 - file path exists
2025-08-09 18:04:08.967 | INFO     | slt_positional_bias.dataset:generate_merged_data_frame:113 - df column 'doc_id' is unique: True
2025-08-09 18:04:08.975 | INFO     | slt_positional_bias.dataset:generate_merged_data_frame:114 - df contains NaN: False


C:\Users\marti\slt_group_2_positional_bias\slt_positional_bias\dataset.py:74: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_filtered_rel = grouped.apply(


,topic_id,doc_id,rel_scoring,topic,doc
0,2024-105741,msmarco_v2.1_doc_56_1565054098#14_3191674270,0,"is it dangerous to have wbc over 15,000 withou...","Hydrocodone Withdrawal: Symptoms, Timeline, an..."
1,2024-105741,msmarco_v2.1_doc_16_287012450#4_490828734,0,"is it dangerous to have wbc over 15,000 withou...",Bacteremia Treatment & Management: Medical Car...
2,2024-105741,msmarco_v2.1_doc_18_512963387#3_575791642,0,"is it dangerous to have wbc over 15,000 withou...",Neonatal sepsis - Wikipedia Neonatal sepsis\nN...
3,2024-105741,msmarco_v2.1_doc_13_1071259551#11_2377754462,0,"is it dangerous to have wbc over 15,000 withou...",Cause of Low and High White Blood Cell Count i...
4,2024-105741,msmarco_v2.1_doc_39_1162159389#10_2348941613,0,"is it dangerous to have wbc over 15,000 withou...",Treatment Options Post CLL Diagnosis | Everyda...
...,...,...,...,...,...
2235,2024-96063,msmarco_v2.1_doc_25_947435380#15_1757291433,0,how using maps can impact your pedagogy,Intersectional Pedagogy | Denver Intersectiona...
2236,2024-96063,msmarco_v2.1_doc_30_727526048#13_1657957899,0,how using maps can impact your pedagogy,Pedagogy: What Educators Need to Know [Plus: 4...
2237,2024-96063,msmarco_v2.1_doc_42_303001194#16_566169799,0,how using maps can impact your pedagogy,The Measures of Academic Progress (MAP) Inform...
2238,2024-96063,msmarco_v2.1_doc_25_785358561#0_1506541701,0,how using maps can impact your pedagogy,Putting Pedagogy Principles to Use in Your Cla...


In [ ]:
API_URL = "https://api.helmholtz-blablador.fz-juelich.de/v1/"
API_KEY = ""
API_MODEL = "alias-large"

from openai import OpenAI

client = OpenAI(
    api_key=API_KEY,
    base_url=API_URL
)

In [18]:
import pandas as pd
from tqdm import tqdm

# Wie viele Dokumente sollen bereinigt werden?
n_to_clean = 2240

# Neue Kopie für bearbeitete Texte
df_cleaned = df_sorted.copy()

# System Prompt
system_prompt = (
    "You are a language model specialized in content distillation. "
    "Your goal is to reduce passages to around 200 tokens while preserving all critical information. "
    "You must avoid removing any content that could be essential to understanding the topic, its implications, or its technical depth. You should use the same vocabularies used in the original document."
)

# Kürzungsschleife
for idx in tqdm(range(n_to_clean)):
    topic = df_sorted.iloc[idx]['topic']
    original_text = df_sorted.iloc[idx]['doc']
    user_prompt = f"""

    Your task is to shorten the passage to approximately 200 tokens while retaining all essential and topic-relevant information. 
    Preserve technical content, facts, processes, and insights mentioned in the text. 
    You should use the vocabularies used in the original document and don't add any new information.

    Avoid:
    - Structural or editorial elements (like tables of contents, section numbers, or headings)
    - Redundant or general-purpose filler text

    Keep:
    - All technical explanations
    - Data and findings
    - Descriptions of methods and applications

    Input passage:
    \"\"\"{original_text}\"\"\"

    Shortened version (~200 tokens, no critical information lost):
    """.strip()

    # LLM Anfrage
    response = client.chat.completions.create(
        model=API_MODEL,
        temperature=0.3,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ],
    )

    raw_output = response.choices[0].message.content.strip()
    if "</think>" in raw_output.lower():
        cleaned_text = raw_output.lower().split("</think>")[-1].strip()
    else:
        cleaned_text = raw_output
    # Gekürzten Text in DataFrame schreiben
    df_cleaned.at[idx, 'doc'] = cleaned_text

store_df_as_parquet(df_cleaned, "output_cleaned_df_eval")



100%|██████████| 2240/2240 [1:53:33<00:00,  3.04s/it]  


2025-08-09 20:37:54.349 | INFO     | slt_positional_bias.dataset:store_df_as_parquet:133 - Data frame saved to C:\Users\marti\slt_group_2_positional_bias\data\processed\output_cleaned_df_eval-1-2025-08-09 20h-37m-54s.parquet


In [ ]:
import pandas as pd
import re
from slt_positional_bias.dataset import load_parquet_as_df, normalize_and_tokenize

def word_count_list(text):
    text = str(text).lower()
    text = re.sub(r'[^\w\s]', '', text) 
    tokens = text.split()
    return tokens

def average_token_count(df):
    token_counts = df["answer"].apply(lambda x: len(normalize_and_tokenize(str(x))))
    return token_counts.mean()

dfs = [
    load_parquet_as_df("LLM-1 - Llama3 405 the best general model and big context size-sample-count-10-1-2025-08-10 00h-18m-01s"), 
    load_parquet_as_df("LLM-1 - Llama3 405 the best general model and big context size-sample-count-20-1-2025-08-10 14h-40m-19s"), 
    load_parquet_as_df("LLM-1 - Llama3 405 the best general model and big context size-sample-count-30-1-2025-08-10 16h-54m-56s"), 
    load_parquet_as_df("LLM-1 - Llama3 405 the best general model and big context size-sample-count-40-1-2025-08-10 18h-46m-08s")
]

avg_list = []

for i, df in enumerate(dfs, start=1):
    avg_tokens = average_token_count(df) 
    avg_wordcount = df["answer"].apply(lambda x: len(word_count_list(x))).mean()  
    avg_list.append({
        "df_num": i,
        "avg_tokens": avg_tokens,
        "avg_wordcount": avg_wordcount
    })

avg_df = pd.DataFrame(avg_list)

print(avg_df)


   df_num  avg_tokens  avg_wordcount
0       1   66.328571     150.807143
1       2   66.675000     154.221429
2       3   64.878571     152.175000
3       4   64.250000     148.414286


In [11]:
df = load_parquet_as_df("LLM-1 - Llama3 405 the best general model and big context size-sample-count-10-1-2025-08-10 00h-18m-01s")
pd.set_option("display.max_colwidth", None)
df["answer"].head(1)

0    Having a WBC count over 15,000 without treatment can lead to severe complications such as bleeding, lung injury, or kidney damage. Treatment may include leukapheresis, blood transfusions, and medicines to manage underlying causes. Without treatment, the risk of complications increases. However, the decision to treat should consider symptom status and disease progression. In some cases, observation may be appropriate if no symptoms are present. It is essential to consult a doctor to determine the best course of action.
Name: answer, dtype: object